In [22]:
import pandas as pd
import numpy as np
import os
import pyarrow
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm

# Testando elasticsearch

In [25]:
es = Elasticsearch(hosts=['http://localhost:9200'])
es.ping()

True

# Importando o dataset

In [2]:
dataset_path = "./archive/amz_br_total_products_data_processed.csv"
df = pd.read_csv(dataset_path)

df.head()

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B0C154VR2L,"SAMSUNG Smart TV Crystal 50"" 4K UHD CU7700 - A...",https://m.media-amazon.com/images/I/71vk2qFDSP...,https://www.amazon.com.br/dp/B0C154VR2L,4.6,269,2279.05,0.0,"TV, Áudio e Cinema em Casa",False,0
1,B0C85J5HYD,Mini Projetor Portatil 5G Wifi 6 Bluetooth 5.0...,https://m.media-amazon.com/images/I/5133wKaOqS...,https://www.amazon.com.br/dp/B0C85J5HYD,4.5,816,409.99,0.0,"TV, Áudio e Cinema em Casa",True,5000
2,B0C1538ZJ4,"Samsung Smart TV Crystal 43"" 4K UHD CU7700 - A...",https://m.media-amazon.com/images/I/71vk2qFDSP...,https://www.amazon.com.br/dp/B0C1538ZJ4,4.7,638,1969.00,2499.0,"TV, Áudio e Cinema em Casa",False,0
3,B0B4V379PC,"Samsung 60BU8000 - Smart TV LED 60' 4K UHD, Wi...",https://m.media-amazon.com/images/I/8110fzpU-P...,https://www.amazon.com.br/dp/B0B4V379PC,4.6,675,3099.00,0.0,"TV, Áudio e Cinema em Casa",False,0
4,B0899BRB4B,Smart TV LED 32'' HD Samsung LH32BETBLGGXZD,https://m.media-amazon.com/images/I/61M025WzQd...,https://www.amazon.com.br/dp/B0899BRB4B,4.7,3945,1079.90,1369.0,"TV, Áudio e Cinema em Casa",True,0


# Tratamento do dataset

In [4]:
columns_to_drop = ['imgUrl', 'productURL']
df.drop(columns_to_drop, axis=1, inplace=True)
df.head()

,asin,title,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B0C154VR2L,"SAMSUNG Smart TV Crystal 50"" 4K UHD CU7700 - A...",4.6,269,2279.05,0.0,"TV, Áudio e Cinema em Casa",False,0
1,B0C85J5HYD,Mini Projetor Portatil 5G Wifi 6 Bluetooth 5.0...,4.5,816,409.99,0.0,"TV, Áudio e Cinema em Casa",True,5000
2,B0C1538ZJ4,"Samsung Smart TV Crystal 43"" 4K UHD CU7700 - A...",4.7,638,1969.00,2499.0,"TV, Áudio e Cinema em Casa",False,0
3,B0B4V379PC,"Samsung 60BU8000 - Smart TV LED 60' 4K UHD, Wi...",4.6,675,3099.00,0.0,"TV, Áudio e Cinema em Casa",False,0
4,B0899BRB4B,Smart TV LED 32'' HD Samsung LH32BETBLGGXZD,4.7,3945,1079.90,1369.0,"TV, Áudio e Cinema em Casa",True,0


In [6]:
df.columns

Index(['asin', 'title', 'stars', 'reviews', 'price', 'listPrice',
       'categoryName', 'isBestSeller', 'boughtInLastMonth'],
      dtype='object')

# Criação de index

In [35]:
if es.indices.exists(index="amazon"):
    response = es.indices.delete(index="amazon")
    print("Índice deletado:", response)
else:
    print("Índice não encontrado.")

Índice não encontrado.


In [8]:
def cria_index_amazon():
    index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "default": {
                    "type": "standard"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "asin": {"type": "keyword"},
            "title": {"type": "text", "analyzer":"default"},
            "stars": {"type": "float"},
            "reviews": {"type": "integer"},
            "price": {"type": "float"},
            "listPrice": {"type": "float"},
            "categoryName": {"type": "text", "analyzer":"default"},
            "isBestSeller": {"type": "boolean"},
            "boughtInLastMonth": {"type": "integer"}
            }
        }
    }

    es.indices.create(index="amazon", body=index_settings)

In [36]:
cria_index_amazon()

# Indexa dados

In [34]:
def index_data(df):
    actions = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Indexando documentos"):
        doc = row.to_dict()
        document = {
            '_index':'amazon',
            '_source': doc
        }
        if(len(actions)>=5):
            
            helpers.bulk(es,actions)
            actions = []
        actions.append(document)
       # es.index(index="amazon", body=document)

In [ ]:
# actions = []
# for file in os.listdir(path):
#     filepath = os.path.join(path,file)
#     doc = get_document_by_xml(filepath)
    
#     document = {
#         '_index': 'petroleo_2',
#         '_source': doc
#     }
#     if(len(actions)>=2000):
#         helpers.bulk(es,actions)
#         actions= []
#     actions.append(document)



In [37]:
index_data(df)

Indexando documentos:   3%|▎         | 41195/1344513 [07:07<3:45:30, 96.32it/s] 


KeyboardInterrupt: 